---
format:
  html:
    code-fold: false
jupyter: python3
---










This block is all about understanding spatial data, both conceptually and practically. Before your fingers get on the keyboard, the following readings will help you get going and familiar with core ideas:

-   [Chapter 1](https://geographicdata.science/book/notebooks/01_geo_thinking.html) of the GDS Book {cite}`reyABwolf`, which provides a conceptual overview of representing Geography in data.

-   [Chapter 3](https://geographicdata.science/book/notebooks/03_spatial_data.html) of the GDS Book {cite}`reyABwolf`, a sister chapter with a more applied perspective on how concepts are implemented in computer data structures.

Additionally, parts of this block are based and source from [Block C](https://darribas.org/gds_course/content/bC/lab_C.html) in the GDS Course {cite}`darribas_gds_course`.

# Hands-on coding

## (Geographic) tables


In [ ]:
import pandas
import geopandas
import xarray, rioxarray
import contextily
import matplotlib.pyplot as plt

### Points

Data If you want to read more about the data sources behind this dataset, head to the [Datasets](./data/datasets) section.

Assuming you have the file locally on the path `./data/`:


In [ ]:
pts = geopandas.read_file("./data/madrid_abb.gpkg")

Sometimes, points are provided as separate columns in an otherwise non-spatial table. For example imagine we have an object `cols` with a column named `X` for longitude and `Y` for latitude. Then, we can convert those into proper geometries by running:

```{{python}}
pts = geopandas.GeoSeries(
    geopandas.points_from_xy(cols["X"], cols["Y"])
)
```


In [ ]:
pts.info()

In [ ]:
pts.head()

#### Lines

Assuming you have the file locally on the path `./data/`: 


In [ ]:
pts = geopandas.read_file("./data/arturo_streets.gpkg")

In [ ]:
lines = geopandas.read_file("./data/arturo_streets.gpkg")

In [ ]:
lines.info()

In [ ]:
lines.loc[0, "geometry"]

#### Polygons


Assuming you have the file locally on the path `./data/`: 


In [ ]:
polys = geopandas.read_file("./data/neighbourhoods.geojson")

In [ ]:
polys = geopandas.read_file("./data/neighbourhoods.geojson")

In [ ]:
polys.head()

In [ ]:
polys.query("neighbourhood_group == 'Retiro'")

In [ ]:
polys.neighbourhood_group.unique()

Assuming you have the file locally on the path `./data/`: 


In [ ]:
sat = xarray.open_rasterio("./data/madrid_scene_s2_10_tc.tif")

In [ ]:
sat

In [ ]:
sat.sel(band=1)

In [ ]:
sat.sel(
    x=slice(430000, 440000),  # x is ascending
    y=slice(4480000, 4470000) # y is descending
)

### Visualisation
 IMPORTANT
You will need version 0.10.0 or greater of `geopandas` to use `explore`. 


In [ ]:
polys.explore()

In [ ]:
polys.plot()

In [ ]:
ax = lines.plot(linewidth=0.1, color="black")
contextily.add_basemap(ax, crs=lines.crs)

```{margin}
See more basemap options [here](https://contextily.readthedocs.io/en/latest/providers_deepdive.html).
```

In [ ]:
ax = pts.plot(color="red", figsize=(12, 12), markersize=0.1)
contextily.add_basemap(
    ax,
    crs = pts.crs,
    source = contextily.providers.CartoDB.DarkMatter
);

In [ ]:
sat.plot.imshow(figsize=(12, 12))

````{margin} IMPORTANT
You will need version 1.1.0 of `contextily` to use label layers. Install it with:


In [ ]:
f, ax = plt.subplots(1, figsize=(12, 12))
sat.plot.imshow(ax=ax)
contextily.add_basemap(
    ax,
    crs=sat.rio.crs,
    source=contextily.providers.CartoDB.VoyagerOnlyLabels,
    zoom=11,
);

### Spatial operations

#### (Re-)Projections

In [ ]:
pts.crs

In [ ]:
sat.rio.crs

In [ ]:
pts.to_crs(sat.rio.crs).crs

In [ ]:
sat.rio.reproject(pts.crs).rio.crs

In [ ]:
# All into Web Mercator (EPSG:3857)
f, ax = plt.subplots(1, figsize=(12, 12))
## Satellite image
sat.rio.reproject(
    "EPSG:3857"
).plot.imshow(
    ax=ax
)
## Neighbourhoods
polys.to_crs(epsg=3857).plot(
    linewidth=2, 
    edgecolor="xkcd:lime", 
    facecolor="none",
    ax=ax
)
## Labels
contextily.add_basemap( # No need to reproject
    ax,
    source=contextily.providers.CartoDB.VoyagerOnlyLabels,
);

#### Centroids

```{margin}
Note the warning that geometric operations with non-project CRS object result in biases.
```

In [ ]:
polys.centroid

In [ ]:
lines.centroid

In [ ]:
ax = polys.plot(color="purple")
polys.centroid.plot(
    ax=ax, color="lime", markersize=1
)

#### Spatial joins

```{margin}
More information about spatial joins in `geopandas` is available on its [documentation page](https://geopandas.org/mergingdata.html#spatial-joins)
```

In [ ]:
sj = geopandas.sjoin(
    lines,
    polys.to_crs(lines.crs)
)

In [ ]:
# Subset of lines
ax = sj.query(
    "neighbourhood == 'Jerónimos'"
).plot(color="xkcd:bright turquoise")

# Subset of line centroids
ax = sj.query(
    "neighbourhood == 'Jerónimos'"
).centroid.plot(
    color="xkcd:bright violet", markersize=7, ax=ax
)

# Local basemap
contextily.add_basemap(
    ax,
    crs=sj.crs,
    source="./data/madrid_scene_s2_10_tc.tif",
    alpha=0.5
)

In [ ]:
sj.info()

#### Areas

In [ ]:
areas = polys.to_crs(
    epsg=25830
).area * 1e-6 # Km2
areas.head()

#### Distances

In [ ]:
cemfi = geopandas.tools.geocode(
    "Calle Casado del Alisal, 5, Madrid"
).to_crs(epsg=25830)
cemfi

In [ ]:
polys.to_crs(
    cemfi.crs
).distance(
    cemfi.geometry
)

In [ ]:
d2cemfi = polys.to_crs(
    cemfi.crs
).distance(
    cemfi.geometry[0] # NO index
)
d2cemfi.head()

In [ ]:
ax = polys.assign(
    dist=d2cemfi/1000
).plot("dist", legend=True)

cemfi.to_crs(
    polys.crs
).plot(
    marker="*", 
    markersize=15, 
    color="r", 
    label="CEMFI", 
    ax=ax
)

ax.legend()
ax.set_title(
    "Distance to CEMFI"
);

## Next steps

If you are interested in following up on some of the topics explored in this block, the following pointers might be useful:

-   Although we have seen here `geopandas` only, all non-geographic operations on geo-tables are really thanks to `pandas`, the workhorse for tabular data in Python. Their [official documentation](https://pandas.pydata.org/docs/) is an excellent first stop. If you prefer a book, McKinney (2012) {cite}`mckinney2012python` is a great one.
-   For more detail on geographic operations on geo-tables, the [Geopandas official documentation](https://geopandas.org/) is a great place to continue the journey.
-   Surfaces, as covered here, are really an example of multi-dimensional labelled arrays. The library we use, `xarray` represents the cutting edge for working with these data structures in Python, and [their documentation](https://xarray.pydata.org/) is a great place to wrap your head around how data of this type can be manipulated. For geographic extensions (CRS handling, reprojections, etc.), we have used `rioxarray` under the hood, and [its documentation](https://corteva.github.io/rioxarray/) is also well worth checking.